In [1]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'C:\Users\seagu\OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'C:\Users\seagu\OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

In [23]:
## Importation des données excel

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
nb_case = len(stepConditions)



# for case in range(nb_case) :
case = 0
if True : 


    hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
    big_it = hyp['big_it']
    T_amb = stepConditions[case]['T_amb']


    # Créer le chemin complet du dossier de résultats
    folder_path_case = os.path.join(folder_path, f"cas_{case+1}")

    # Vérifier si le dossier n'existe pas déjà
    if not os.path.exists(folder_path_case):
        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_path_case)

    hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

    file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
    file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
    file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
    file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'

    Elapsed_time = pd.DataFrame(columns=['iteration', 'time'])

    # File > Application > Start > folder_path_ansys
    # Ensuite copier le numéro du server et le modifier ci-dessous
    solver_path = folder_path_ansys + '\server\\' + 'server_info-16316.txt'
    # Créer une session
    solver = pyfluent.connect_to_fluent(server_info_filepath=solver_path)
    tui = solver.tui



    ## Modification des paramètres invariables

    # On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
    theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
    jg.change_gravity(tui, theta)

    # On initialise la température ambiante
    jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

    # On initialise le critère de convergence pour les transferts thermiques
    convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
    jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')


    ## Première initialisation des paramètres dans ANSYS Fluent

    for i in range(len(Inputs_PyFluent)):
        named_expression = Inputs_PyFluent.iloc[i]['named_expression']
        value = Inputs_PyFluent.iloc[i]['value']
        unit = Inputs_PyFluent.iloc[i]['unit']
        jg.change_named_expression(tui, named_expression, value, unit)


    # Initialisation des valeurs avec PVT-thermal-performances-model

    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

    # On initialise la première température avec le modèle et le transfert radiatif
    apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, stepConditions[case]['T_fluid_in0'])

    e_PV = 2.5e-3
    volume_PV = 1134*1708*e_PV*1e-6
    Heat_Generation_Rate = -df_one['Qdot_top_rad'].values[0]/volume_PV
    apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

    # On initialise les a_f et b_f avec le modèle
    for i in range(1, nb_hx + 1):
        apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
        apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])


    # Met à jour les méthodes dans les hypothèses
    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'


    ## Initialisation
    jg.standard_initialization(tui, 'cd_fc', 0)


    limit_big_it = 5 # Nombre d'itérations maximum
    convergence_criteria = 1e-1 # Critère de convergence pour le transfert thermique
    nb_it = 30 # Nombre d'itération lancée sur ANSYS
    iteration = 0

    while iteration < limit_big_it :
        
        time_start = time.time()

        # try : 
        ## Calcul des températures

        apb.fill_initial_temperature_dict(Inputs_PyFluent)



        ## Sauvegarde des données du modèle 1D
        # Sauvegarde de df_one
        keys = list(res['part1']['df_one'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for i in range(1, nb_hx+3):
            part_key = f'part{i}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['df_one'][key])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_df_one = pd.DataFrame(values_dict)

        # Ajouter les noms des index pour refléter les parties
        result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
        result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

        # Sauvegarde de slices_df
        keys = list(res['part1']['slices_df'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for part in range(1, nb_case+3):
            part_key = f'part{part}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['slices_df'][key])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_slices_df = pd.DataFrame(values_dict)
        result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)





        ## Modification des paramètres dans ANSYS Fluent
        apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, stepConditions[case]['T_fluid_in0']) # Au cas où
        for part in range(1, nb_hx+1):
            T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
            a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
            b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
            jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
            jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
            jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
        T_fluid_out = apb.get_value(f'T_fluid_in_{nb_hx+1}', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, f'T_fluid_in_{nb_hx+1}', T_fluid_out, 'K')
        jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')



        ## Lancement de la simulation

        solver.solution.run_calculation.iterate(number_of_iterations=nb_it)
    



        jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
        jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

        Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
        Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

        Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
        Qdot_conv['Component'] = Qdot_tot['Component']
        Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

        Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

        hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
            'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

        ## Partie très moche, difficile de faire autrement pour le moment
        part1 = ['manifold_yu']
        part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
        part3 = ['hx_flat_yu_air']
        part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
        part5 = ['hx_flat_yd_air']
        part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
        part7 = ['manifold_yd']

        part1_top = []
        part2_top = []
        part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
        part4_top = []
        part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
        part6_top = []
        part7_top = []


        Areas_top = []
        Areas_back = []
        for part in panelSpecs['decomp'].keys() :
            if part != 'main':
                Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
                Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


        ## Sauvegarde des données du modèle CFD
        phi_top_list = []
        phi_tube_list = []
        phi_abs_list = []

        for i in range(1, nb_hx+3) :
            if i == 3 or i == 5 :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
                phi_abs_list.append(1e-6)
            else :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(1e-6)
                phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

        result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
        result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
        result_CFD['phi_top'] = phi_top_list 
        result_CFD['phi_abs'] = phi_abs_list
        result_CFD['phi_tube'] = phi_tube_list 



        ## Envoi des flux au PVT-thermal-performance-model
        result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
        Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)


        # Pour permettre au modèle de ne pas recommencer depuis le début
        hyp['specific_init'] = df_one['T_PV'].values[0]

        ## Appel du PVT-thermal-performances-model

        df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,stepConditions[case],hyp)

        Heat_Generation_Rate = -df_one['Qdot_top_rad'].values[0]/volume_PV
        apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

        # On initialise les a_f et b_f avec le modèle
        for i in range(1, nb_hx + 1):
            apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
            apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



        ## On incrémente 

        big_it = big_it + 1
        hyp['big_it'] = big_it
        file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
        file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
        file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
        file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


        print(f'Cas {case+1} iteration {big_it} / {limit_big_it} terminée')
        iteration += 1
        
        # except Exception as e:
        #     big_it = big_it + 1
        #     print(f'Cas {case} iteration {big_it} / {limit_big_it} non aboutie : {e}')
        #     iteration = limit_big_it
        #     continue
        
        jg.write_data(tui, folder_path_case, f'data_cas{case+1}_it{big_it}')
        jg.write_residuals_file(tui, folder_path_case, f'residuals_cas{case+1}_it{big_it}')

        time_end = time.time()
        Elapsed_time = Elapsed_time.append({'iteration': big_it, 'time': time_end - time_start}, ignore_index=True)
    
    Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case+1}.csv'),sep=';', index=False)

KeyboardInterrupt: 

In [ ]:
## Remettre les conditions initiales d'un cas à une certaine itération

mesh = 0
case = 0
iteration = 0

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

folder_mesh = f'Simu_M{mesh+1}'
folder_path_mesh = os.path.join(folder_path, folder_mesh)
folder_name = f'cas_{case}'
folder_path_case = os.path.join(folder_path_mesh, folder_name)
hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')
file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'
df_PyFluent = pd.read_csv(file_path_Inputs_PyFluent, sep=';')


Inputs_PyFluent = df_PyFluent[mesh][case-1][iteration]

for i in range(len(Inputs_PyFluent)):
        named_expression = Inputs_PyFluent.iloc[i]['named_expression']
        value = Inputs_PyFluent.iloc[i]['value']
        unit = Inputs_PyFluent.iloc[i]['unit']
        jg.change_named_expression(tui, named_expression, value, unit)